# Model construction

# 0. Import Data and Modules

In [136]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
import xgboost as xgb
from sklearn import metrics
from datetime import datetime
import random
import seaborn as sns

In [137]:
#os.chdir("C:/Users/TonyG/Documents/GitHub/bads/kaggle") # Tony
#os.chdir("C:/Users/erin-/Documents/bads/kaggle") # Erin
os.chdir("C:/Users/gotschat/Documents") # Server
data = pd.read_pickle('./data/known_cleaned_w_dummies')
X = data.drop("return", axis = 1)
y = data["return"]
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98928 entries, 1 to 100000
Columns: 132 entries, item_id to user_state_Thuringia
dtypes: float64(3), int64(4), uint8(125)
memory usage: 17.8 MB


### For Preliminary version only: Create test and train sets based on the known dataset via random splitting

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 314)

## 1. Boosted Trees 

### 1.1 with Random Search CV and custom score function

In [139]:
def cost_custom_score(estimator, X, y):
    zeros = np.zeros(y.shape[0])
    hat = 1 * estimator.predict(X)
    val = X.loc[:, "item_price"]
    score = -1 * (np.fmax(np.array(y - hat), zeros) * (0.5 + 0.1 * val) +
                 np.fmax(np.array(hat - y), zeros) * (0.5 * val))
    return score.sum()

In [140]:
def cost_custom_eval(y_true, y_test, X, relative = False):
    '''Custom Evaluation based on cost matrix
    relative: True: return TP, FP, FN, TN, False: return the aggregated score, i.e. TP + TN + FP + FN
    '''
    y_true = 1 * y_true # Convert to integer
    y_test = 1 * y_test
    
    zeros = np.zeros(y_true.shape[0]) # zeros for calculation
    val = X.loc[:, "item_price"] # item_price of item 
    
    TP, TN = np.array([0, 0]) # TP and TN are always 0
    
    # Calculate FP
    FP = np.fmax(np.array(y_test - y_true), zeros) * (0.5 * val)
    
    # Calculate FN
    FN = np.fmax(np.array(y_true - y_test), zeros) * (0.5 + 0.1 * val)
    
    # If clause for relative parameter
    if(relative == True):
        return(np.array([TP, FP.sum(), FN.sum(), TN]))
    else:
        return(np.sum([FP, FN]))
    
    #score = 1 * (np.fmax(np.array(y_true - y_test), zeros) * (0.5 + 0.1 * val) +
    #             np.fmax(np.array(y_test - y_true), zeros) * (0.5 * val))
    #return score.sum()

### Reference Model Performance

In [141]:
# Train a simple boosted tree classifier and calculate the custom score for this model using the X_test and y_test hold out
ref_mod = xgb.XGBClassifier(objective = "binary:logistic")
ref_mod.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:12:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=40, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [142]:
base_perf = cost_custom_eval(y_test, 1 * ref_mod.predict(X_test), X_test)
base_perf

175353.275

### Cross-Validation

In [143]:
fixed_boosting_rnds = True # Implemented a switch whether number of boosting rounds shoul be tuned as well
param_grid = {"max_depth" : np.arange(5,80, step = 5), # maximum number of nodes for each base learner
             "eta" : stats.uniform(0.05, 0.70), # Learning rate
             "gamma" : stats.uniform(0.05, 3), # Minimum node impurity gain required to attempt a split
             "lambda" : stats.uniform(0, 8), # L2 regularization coefficent
             "colsample_bytree" : np.arange(0.2, 1, step = 0.1), # proportion of features considered for each base learner
             "subsample" : np.arange(0.5, 1, step = 0.1), # proportion of training data subsampled for each base learner
             "n_estimators" : np.where(fixed_boosting_rnds, 20, np.arange(10, 40, step = 5))} # Number of boosting rounds; will be inreased for the winner
    
gbm = xgb.XGBClassifier(objective = "binary:logistic") # initiate estimator
metric = cost_custom_score # Scoring metric used during training
n = 600 # number of candidates to consider
fold = 5 # number of folds for cv
n * fold # number of fits

3000

In [ ]:
randomized_auc = RandomizedSearchCV(estimator = gbm, n_iter = n, cv = fold, scoring = metric,
                                    param_distributions = param_grid, verbose = 1, random_state = 321,
                                   n_jobs = -1, refit = True)
randomized_auc.fit(X_train,y_train)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.


In [ ]:
print("Best Parameters:", randomized_auc.best_params_)
print("Best Score:", randomized_auc.best_score_ * (-1))

If trained on server:

In [ ]:
# copy paste saved dictionary if not on server
# params =

params = randomized_auc.best_params_

eval_set = [(X_train, y_train), (X_test, y_test)] # Evaluation set for early stopping
randomized_auc = xgb.XGBClassifier(objective = "binary:logistic",
                                   max_depth = params["max_depth"],
                                   learning_rate =  params["eta"],
                                   gamma = params["gamma"],
                                   reg_lambda = params["lambda"],
                                   colsample_bytree = params["colsample_bytree"],
                                   subsample = params["subsample"],
                                   n_estimators = 200, # Train with higher number of boosting rounds as in the cv part
                                   random_state = 321)
randomized_auc.fit(X_train, y_train, early_stopping_rounds = 10, eval_metric = "auc", eval_set = eval_set,
                  verbose = True) # Enable earlier stopping when within 10 rounds no improvement is observed using auc as evaluation metric

In [ ]:
randomized_auc.best_iteration, randomized_auc.best_score

Display evaluation performance during training

In [ ]:
from matplotlib import pyplot
results = randomized_auc.evals_result()
epochs = len(results['validation_0']['auc'])
x_axis = range(0, epochs)
# plot roc
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['auc'], label='Train')
ax.plot(x_axis, results['validation_1']['auc'], label='Test')
ax.legend()
pyplot.ylabel('ROC')
pyplot.title('XGBoost Classification Error')
pyplot.show()

Display custom score using the tuned model and the hold out test set and compare with base performance

In [ ]:
preds = randomized_auc.predict(X_test)
print("Tuned Custom Evaluation Score:", cost_custom_eval(y_test, 1 * preds, X_test).round(5))
print("Base Custom Evaluation Score:", base_perf.round(5))
print("Tuned AUC score:", metrics.roc_auc_score(y_test, randomized_auc.predict(X_test)).round(5))
print("Base AUC Score:", metrics.roc_auc_score(y_test, ref_mod.predict(X_test)).round(5))
print("Monkey Custom Evaluation Score:", cost_custom_eval(y_test, 1* random.choices(np.array([True, False]), k = y_test.shape[0]), X_test).round(5))
print("Monkey AUC Score:", metrics.roc_auc_score(y_test, random.choices(np.array([True, False]), k = y_test.shape[0])).round(5))

Return feature importance

In [134]:
quant = pd.Series(randomized_auc.feature_importances_,
                  index = X_train.columns).sort_values(ascending = False).quantile(q = 0.7) # get 70% quantile
importance_trained = pd.Series(randomized_auc.feature_importances_, index = X_train.columns).sort_values(ascending = False)
importance_trained[importance_trained >= quant] # Display only the 30% highest importance values

item_size_unsized                    0.075183
time_to_delivery                     0.054843
item_color_pallid                    0.034539
item_color_?                         0.019875
item_price                           0.017996
item_size_Other                      0.012893
item_color_dark denim                0.012059
item_color_floral                    0.011925
item_id                              0.011783
item_color_coral                     0.011368
brand_id                             0.011221
item_color_silver                    0.011153
item_color_navy                      0.010941
item_size_6                          0.010378
customer_age                         0.009335
user_id                              0.009224
user_title_Mrs                       0.009187
item_color_habana                    0.008928
item_color_Other                     0.008898
item_size_116                        0.008854
user_age                             0.008827
item_size_l                       

In [135]:
quant = pd.Series(randomized_auc.feature_importances_,
                  index = X_train.columns).sort_values(ascending = False).quantile(q = 0.3) # get 70% quantile
importance_trained = pd.Series(randomized_auc.feature_importances_, index = X_train.columns).sort_values(ascending = False)
importance_trained[importance_trained < quant] # Display only the 30% lowest importance values

item_color_aubergine        0.005363
user_state_Saxony-Anhalt    0.005343
item_color_ancient          0.005314
item_size_35                0.005301
item_color_orange           0.005283
item_color_yellow           0.005264
item_color_azure            0.005154
item_color_striped          0.005153
item_size_50                0.005136
item_size_xs                0.005118
item_size_40+               0.005011
item_size_23                0.004774
item_size_3                 0.004752
item_size_152               0.004724
item_size_32                0.004651
item_size_34                0.004514
item_size_41+               0.004494
item_size_42+               0.004445
item_size_25                0.004379
item_size_9+                0.004364
item_size_4+                0.004252
item_size_13                0.004189
item_size_27                0.004165
item_color_gold             0.004017
item_size_24                0.003996
item_size_26                0.003846
item_color_mahagoni         0.003762
i

Save AUC score and custom score into .txt file

In [90]:
perf_auc = metrics.roc_auc_score(y_test, randomized_auc.predict(X_test))
perf_cust = cost_custom_eval(y_test, 1 * preds, X_test)
sys_time = datetime.now().strftime("%d-%b-%Y (%H:%M)")

In [22]:
%%script false --no-raise-error
# Initiate .txt and column header, is run once
f = open("results_RS.txt", "w+")
file = open("results_RS.txt", "a")
file.write("\t".join(["Time", "\t\t AUC", "\t Custom Score", "\n"]))
file.close()

Couldn't find program: 'false'


In [91]:
file = open("results_RS.txt", "a")
file.write(" ".join([sys_time,"\t", str(perf_auc.round(5)),"\t", str(perf_cust.round(5)), "\n"]))
file.close()

Display best parameters and best score during training

In [92]:
print("Best Parameters:", randomized_auc.get_params())

Best Parameters: {'objective': 'binary:logistic', 'use_label_encoder': True, 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.2, 'gamma': 2.176864565876066, 'gpu_id': -1, 'importance_type': 'gain', 'interaction_constraints': '', 'learning_rate': 0.0715808240526306, 'max_delta_step': 0, 'max_depth': 55, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 200, 'n_jobs': 40, 'num_parallel_tree': 1, 'random_state': 321, 'reg_alpha': 0, 'reg_lambda': 3.1622167750579484, 'scale_pos_weight': 1, 'subsample': 0.8999999999999999, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}


Save hyperparameters inside a .txt file

In [ ]:
%%script false --no-raise-error
# Initiate .txt and column header, is run once
f = open("hyperparams.txt", "w+")
file = open("hyperparams.txt", "a")
file.write("Hyperparameters in Dictionary format \n")
file.close()

In [93]:
file = open("hyperparams.txt", "a")
file.write("\t".join([sys_time, str(randomized_auc.get_params()), "\n"]))
file.close()

Create Confusion matrix with probabilites and a custom confusion matrix based on the cost matrix

In [94]:
metrics.confusion_matrix(y_test, randomized_auc.predict(X_test), normalize = "all").round(5) # TP FP // FN TN

array([[0.37845, 0.16456],
       [0.15016, 0.30683]])

In [95]:
def cust_confusion_matrix(y_true, y_pred, X, normalize = True, rnd = False):
    '''Custom Confusion Matrix with cost matrix
    y_true: 1-d array bool
    y_pred: 1-d array bool
    X: n-d array of data which includes the item price for the respective observations (float)
    normalize: True: Should the custom score be divided by the possible maximum costs?
    rnd: True: Should the normalizing value be calculated using a monkey guessing at random which item will be returned?
    '''
    y_true = 1 * y_true # convert to integer
    y_pred = 1 * y_pred # convert to integer
    
    # Calculate TP, FP, FN and TN
    TP, FP, FN, TN = cost_custom_eval(y_true, y_pred, X, True)
    
    # If clause for random normalization
    if(normalize == True and rnd == True):
        # custom prediction 
        y_random_pred = random.choices(np.array([False, True]), k = y_true.shape[0])
        FP_rel, FN_rel = cost_custom_eval(y_true, y_random_pred, X, True)[1:3] # retrieve FP rate for this case
        results = np.array([[TP, FP/FP_rel],
                            [FN/FN_rel, TN]])
        return(results)
    elif(rnd == False):
        # If clause for normalize
        if(normalize == True):
            y_pred_rel_fp = np.ones(y_true.shape[0]) # Every observation is classified as one
            FP_rel = cost_custom_eval(y_true, y_pred_rel_fp, X) # retrieve FP rate for this case

            y_pred_rel_fn = np.zeros(y_true.shape[0]) # Every observation is classified as zero
            FN_rel = cost_custom_eval(y_true, y_pred_rel_fn, X) # retrieve FN rate for this case
            results = np.array([[TP, FP/FP_rel],
                                [FN/FN_rel, TN]])
            return(results)
        else:
            results = np.array([[TP, FP],
                                [FN, TN]])
            return(results)
    
    

Intepretation of a relative cost matrix based confusion matrix

In [96]:
cust_confusion_matrix(y_test, preds, X_test, True)

array([[0.        , 0.40160672],
       [0.2652876 , 0.        ]])

The entry [1,2] describes how our model performs compared to a model which classifies all purchases as returns. The lower this value the lower are the missed profits from not discouraging the customer to place their order. Conversely, the entry [2,1] describes how our model performed compared to a model which flags every purchase as an item which will not be returned. The lower this values the less costs occur from handling the return. 

Finally, we can compare our model in this manner with a model that randomly flags whether an item is returned using the parameter *rnd*.

In [97]:
cust_confusion_matrix(y_test, preds, X_test, True, True)

array([[0.        , 0.79794665],
       [0.53712908, 0.        ]])

Create .csv of predictions for unknown data set

In [10]:
data_u = pd.read_pickle('./data/unknown_cleaned_w_dummies')

In [11]:
preds = randomized_auc.predict_proba(data_u)[:, 1]
predict_unknown = pd.Series(preds, index=data_u["item_id"].index, name='return')
predict_unknown.to_csv("first_pred.csv")

### 1.2 With random search CV and combined model instance

First subset the training set into three categories $C_j$, $j \in \{1,2,3\}$, with $C_j = \{X_i : X_{il} \in I_j\}$, where $X_i$ are the observations, $X_{il}$ is the item price for the respective observation and $I_j$ are item price intervals, namely 
\begin{align*}
I_1 &= [0, K_1) \\
I_2 &= [K_1, K_2] \\
I_3 &= (K_2, \infty).
\end{align*}
The thresholds $K_1$ and $K_2$ are given by the 0.25 and 0.75 quantiles for the distribution in the training set of item price.
Afterwards train three different boosted classifier on those subsets using 5-fold crossvalidation. 
Finally, create a custom class which incorporates all three models and uses a customized predict attribute to use only the relevant model based on the item price of the test observation.

#### Split trainingsset in three subsets

In [8]:
# Get quantiles 
K1, K2 = X_train["item_price"].quantile([0.25, 0.75])

# Create three subsets of training data
C1 = X_train[X_train["item_price"] < K1]
C2 = X_train.loc[(X_train["item_price"] >= K1) & (X_train["item_price"] <= K2), :]
C3 = X_train[X_train["item_price"] > K2]

# Create the according target vectors
Y1 = y_train[C1.index]
Y2 = y_train[C2.index]
Y3 = y_train[C3.index]

Train three different models using 5-fold crossvalidation using only the subsets

In [10]:
# Grid stays the same for all three models
param_grid = {"max_depth" : np.arange(5,80, step = 5),
             "eta" : stats.uniform(0.1, 0.8),
             "gamma" : stats.uniform(0.05, 3),
             "lambda" : stats.uniform(0, 5),
             "colsample_bytree" : np.arange(0.2, 1, step = 0.1),
             "subsample" : np.arange(0.5, 1, step = 0.1),
             "n_estimators" : np.arange(10, 80, step = 5)}

# Instantiate classifier
gbm = xgb.XGBClassifier(objective = "binary:logistic")

metric = cost_custom_score # Custom scoring function, see 2.1
n = 300 # Number of search iterations
fold = 5 # 5-fold CV

In [11]:
# Subset C1
mod_c1 = RandomizedSearchCV(estimator = gbm, n_iter = n, cv = fold, scoring = metric,
                                    param_distributions = param_grid, verbose = 1, random_state = 321,
                                   n_jobs = -1)
mod_c1.fit(C1, Y1) 

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 55.2min
[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed: 136.3min
[Parallel(n_jobs=-1)]: Done 720 tasks      | elapsed: 269.4min
[Parallel(n_jobs=-1)]: Done 1170 tasks      | elapsed: 422.5min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 531.7min finished
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[04:22:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { early_stopping_rounds, num_boost_round } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:23:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=10, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constrain...
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000A2E3F1AA60>,
                                        'lambda': <scipy

In [29]:
# Save Parameter dictionary
mod_c1_params = mod_c1.best_params_
with open('mod_c1_params.txt', 'w') as f:
    print(mod_c1_params, file=f)

In [12]:
# Subset C2
mod_c2 = RandomizedSearchCV(estimator = gbm, n_iter = n, cv = fold, scoring = metric,
                                    param_distributions = param_grid, verbose = 1, random_state = 321,
                                   n_jobs = -1)
mod_c2.fit(C2, Y2) 

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 139.7min
[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed: 321.3min
[Parallel(n_jobs=-1)]: Done 720 tasks      | elapsed: 549.9min
[Parallel(n_jobs=-1)]: Done 1170 tasks      | elapsed: 831.7min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 1043.4min finished
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:46:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { early_stopping_rounds, num_boost_round } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:46:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=10, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constrain...
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000A2E3F1AA60>,
                                        'lambda': <scipy

In [28]:
# Save Parameter dictionary
mod_c2_params = mod_c2.best_params_
with open('mod_c2_params.txt', 'w') as f:
    print(mod_c2_params, file=f)

In [13]:
# Subset C3
mod_c3 = RandomizedSearchCV(estimator = gbm, n_iter = n, cv = fold, scoring = metric,
                                    param_distributions = param_grid, verbose = 1, random_state = 321,
                                   n_jobs = -1)
mod_c3.fit(C3, Y3) 

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 47.4min
[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed: 123.0min
[Parallel(n_jobs=-1)]: Done 720 tasks      | elapsed: 208.4min
[Parallel(n_jobs=-1)]: Done 1170 tasks      | elapsed: 377.0min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 452.9min finished
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:19:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { early_stopping_rounds, num_boost_round } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[05:19:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=10, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constrain...
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000A2E3F1AA60>,
                                        'lambda': <scipy

In [27]:
# Save Parameter dictionary
mod_c3_params = mod_c3.best_params_
with open('mod_c3_params.txt', 'w') as f:
    print(mod_c3_params, file=f)


In [33]:
mod_c3.cv_results_

{'mean_fit_time': array([1086.39582286, 2139.23285995,  892.68328056,  938.22382946,
        1911.38577652, 1484.85955844, 1007.4564424 ,  132.0170022 ,
         740.68059134,  959.93398142,  184.71005397, 1408.77953815,
         474.21984429,  132.18336306,  198.52198334,  665.92604961,
        1730.91441336,  777.58360434,  732.35615788,  331.56497569,
         166.16255465,  170.91654234,  291.70691071,  710.63079195,
         993.3320117 ,  677.83053808, 1633.99207878,  948.49874563,
         844.82669735,  840.64846525,  289.54442396, 1873.13045855,
        1625.66163955, 1815.95087781,  221.20467286,  763.78889213,
         502.93275366,  293.61815529,  366.0282342 ,  383.46646271,
         512.95542603,  717.21434555,  408.36148791, 1563.88642373,
         287.43598909,  668.26690879,  900.55888138,  410.16602941,
         221.1534234 ,  273.03789406, 1174.0790411 ,  324.87531991,
         361.56072268,  325.77098417,  370.26111913,  163.13002815,
         322.54887471,  618.900

Create custom model class

In [15]:
class CustomMod:
    def __init__(self, model_1, model_2, model_3, inter):
        self.mod1 = model_1
        self.mod2 = model_2
        self.mod3 = model_3
        self.thresholds = [inter[0], inter[1]]
        
    def predict_custom(self, X, val = "item_price"):
        '''Custom Predict function
        
        '''
        # Split into subsets and store index
        C1 = X.loc[X[val] < self.thresholds[0], :].index
        C2 = X.loc[(X[val] >= self.thresholds[0]) &
                   (X[val] <= self.thresholds[1]), :].index
        C3 = X.loc[X[val] > self.thresholds[1], :].index
        index_cust = C1.append([C2, C3]) # Index for consistent prediction order
        
        # Predict using the three models
        Y1 = self.mod1.predict(X.loc[C1, :]) # Model 1
        Y2 = self.mod2.predict(X.loc[C2, :]) # Model 2
        Y3 = self.mod3.predict(X.loc[C3, :]) # Model 3
        
        # Combine prediction to be consistent with the order of X
        preds = pd.Series(np.concatenate([Y1, Y2, Y3]), name = "predictions", index = index_cust) # Create series of predictions
        preds = preds[X.index] # Reorder based in the index of X
        
        return(preds)


In [23]:
test = CustomMod(mod_c1.best_estimator_, mod_c2.best_estimator_, mod_c3.best_estimator_, inter = [K1,K2])
test.predict_custom(X_train)

order_item_id
65902    False
90887    False
65284     True
31170     True
74301     True
         ...  
11239    False
43483     True
45333    False
76278    False
15059    False
Name: predictions, Length: 79142, dtype: bool

In [20]:
X_test, preds

(               item_id  brand_id  item_price  user_id  time_to_delivery  \
 order_item_id                                                             
 3548                84         5       24.90    20671               NaN   
 14074               70        34       19.90    33217               4.0   
 38997             1517         6       69.90    15564              21.0   
 98241             1692        50       89.90    40112               2.0   
 46914             1625        20       99.90    16535              22.0   
 ...                ...       ...         ...      ...               ...   
 15325              159        54       39.90    31764               NaN   
 21001              699        39       44.95    34279               3.0   
 71194             1609         1      179.90    16864               4.0   
 83590               16        12       69.90    26046               2.0   
 50164             1593        17      219.90     5006               NaN   
 
          

In [24]:
preds = test.predict_custom(X_test)
print("Custom Evaluation Score:", cost_custom_eval(y_test, 1 * preds, X_test))
print("Base Custom Evaluation Score:", base_perf)

Custom Evaluation Score: 160620.02300000002
Base Custom Evaluation Score: 175109.465


In [ ]:
param_grid = {"max_depth" : np.arange(1,20),
             "eta" : stats.uniform(0.1, 0.8),
             "gamma" : stats.uniform(0.05, 3),
             "lambda" : stats.uniform(0, 5),
             "alpha" : stats.uniform(0, 5),
             "colsample_bytree" : np.arange(0.2, 1, step = 0.1),
             "subsample" : np.arange(0.5, 1, step = 0.1),
             "n_estimators" : np.arange(10, 50, step = 5)}
    
gbm = xgb.XGBClassifier(objective = "binary:logistic", 
             num_parallel_tree = 1, num_boost_round = 80, early_stopping_rounds = 10)
#metric = "roc_auc"
n = 200
fold = 5

In [ ]:
randomized_auc = RandomizedSearchCV(estimator = gbm, n_iter = n, cv = fold, scoring = cost_custom_score,
                                    param_distributions = param_grid, verbose = 1, random_state = 321,
                                   n_jobs = -1)
randomized_auc.fit(X,y)

### 1.3 with Bayesion optimization CV

In [42]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 123)

dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)

Define function to use Bayesian optimization to optimize a given function on a given dataset based on given parameters

In [49]:
def xgb_evaluate(max_depth, gamma, eta, colsample_bytree, lam, alph, est):
    params = {'eval_metric': 'auc',
              'max_depth': int(max_depth),
              'eta': eta,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              'objective' : 'binary:logistic',
              'n_estimators' :  int(est),
              'lambda' : lam,
              'alph' : alph}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round = 150, nfold = 10, early_stopping_rounds = 30,
                      nthread = 5)
    return cv_result['test-auc-mean'].iloc[-1]

In [50]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 200), 
                                             'gamma': (0, 8),
                                             'colsample_bytree': (0.3, 0.9),
                                             'eta' : (0.001, 0.3),
                                             'lam' : (0.1, 8),
                                             'alph' : (0.1, 3),
                                             'est' : (10, 80)})
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points = 5, n_iter = 2)

|   iter    |  target   |   alph    | colsam... |    est    |    eta    |   gamma   |    lam    | max_depth |
-------------------------------------------------------------------------------------------------------------


TypeError: cv() got an unexpected keyword argument 'nthread'

Extract best parameters

In [5]:
params = xgb_bo.max
#params['max_depth'] = int(params['max_depth'])
params["params"]["max_depth"] = int(params["params"]["max_depth"])
#params["params"]["n_estimators"] = int(params["params"]["est"])
params["params"]['objective'] = 'binary:logistic'
params["params"].pop("est")
params

NameError: name 'xgb_bo' is not defined

In [10]:
params =  {'alpha': 0.5793037571735223,

  'colsample_bytree': 0.3,

  'eta': 0.11,

  'gamma': 0.0,

  'lambda': 6.468522967337128,

  'max_depth': 77,

  'objective': 'binary:logistic'}

 Train model with optimal parameters and calculate auc for test set

In [11]:
model_opt = xgb.train(params, dtrain, num_boost_round = 400)

preds = model_opt.predict(dtest)
roc_auc_score(y_test, preds)

[13:27:36] WARNING: D:\Build\xgboost\xgboost-1.3.1.git\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7745377371183007

Create .csv of prediction for unknown dataset

In [12]:
data_u = pd.read_pickle('./data/unknown_cleaned_w_dummies')

In [13]:
preds = model_opt.predict(xgb.DMatrix(data_u))
predict_unknown = pd.Series(preds, index=data_u["item_id"].index, name='return')
predict_unknown.to_csv("sixth_pred.csv")